In [1]:
import numpy as np
import numba
from numba import jit

In [19]:

# @jit(nopython=True, parallel=True)
def compute_intersection_single_element(x1,y1, x2,y2, x3,y3, x4,y4, 
                                        delta_x_tolerance=1e-6, grad_diff_tolerance=1e-6,
                                       verbose=False):
    
    
    inf_slope_status = np.array([0,0], np.int32)
    parallel_flag = False # for readability
    # check line 1 conditions
    dx_line_1 = x2 - x1
    dy_line_1 = y2 - y1
    if np.abs(dx_line_1) < delta_x_tolerance:
        inf_slope_status[0] = 1
    
    # check line 2 conditions
    dx_line_2 = x4 - x3
    dy_line_2 = y4 - y3
    if np.abs(dx_line_2) < delta_x_tolerance:
        inf_slope_status[1] = 1
    
    # if both have inf slope, they are parallel
    parallel_flag = (np.sum(inf_slope_status) == 2)
    if parallel_flag:
        if verbose:
            print("compute_intersection_single_element: Lines are parallel and have inf slope")
        return (-9999,-9999) # this value is chosen as it will be outside 
                             #  of our detection range (to be filtered later)
    
    # since one is inf and the other is not, they are not parallel
    if (np.sum(inf_slope_status) > 0):
        # there is one line that has inf slope
        if (inf_slope_status[0] == 1): # line one has infinite slope
            x = x1 # pick either one point x1 or x2 as they are the "equal"
            y = (x - x3)*(y4-y3) / (x4-x3) + y3
            if verbose:
                print("compute_intersection_single_element: Line1 has inf slope")
            return (x, y)
        
        if (inf_slope_status[1] == 1):
            x = x3 # pick either one point x3 or x4 as they are the "equal"
            y = (x - x1)*(y2-y1) / (x2-x1) + y1
            if verbose:
                print("compute_intersection_single_element: Line2 has inf slope")
            return (x, y)
            
    else:
        
        # check both line conditions
        m1 = dy_line_1 / dx_line_1
        m2 = dy_line_2 / dx_line_2

        # Note: if the two lines are collinear, they are parallel
        if np.abs(m1 - m2) < grad_diff_tolerance:
            parallel_flag = True


        if parallel_flag:
            if verbose:
                print("compute_intersection_single_element: Lines are parallel")
            return (-9999,-9999) # this value is chosen as it will be outside 
                                 #  of our detection range (to be filtered later)

        if verbose:
            print("compute_intersection_single_element: No line has inf slope")

        x = (m2 *x4 - m1* x2 - y4 + y2) / (m2 - m1)
        y = m2 * (x - x4) + y4
        return (x, y)
    
    raise ValueError # np.sum(inf_slope_status) is abnormal
    

In [33]:
# from PointPillar
def compute_intersection_single_element(x1,y1, x2,y2, x3,y3, x4,y4, verbose=True):
    num = (x1*y2 - y1*x2) * (x3-x4) - (x1-x2) * (x3*y4 - y3*x4);
    den = (x1-x2) * (y3-y4) - (y1-y2) * (x3-x4);
    x = num/(den + 1e-6)
    
    num = (x1*y2 - y1*x2) * (y3-y4) - (y1-y2) * (x3*y4 - y3*x4);
    den = (x1-x2) * (y3-y4) - (y1-y2) * (x3-x4);
    y = num/(den + 1e-6)
    return (x,y)
    

In [34]:
# test the compute_intersection_single_element:
# test cases:
#     1. two lines have inf slope, both lines are parallel
#     2. no line has inf slope, both lines are parallel
#     3. two lines intersect internally, both lines are not parallel, no line has inf slope
#     4. two lines intersect externally, both lines are not parallel, no line has inf slope
#     5. two lines intersect internally, both lines are not parallel, 1 line has inf slope
#     6. two lines intersect externally, both lines are not parallel, 1 line has inf slope
#     7. two lines intersect internally, both lines are not parallel, one line has 0 slope
#     8. two lines intersect externally, both lines are not parallel, one line has 0 slope
#     9. two lines intersect internally, both lines are not parallel, 1 line has inf slope, 1 line has 0 slope
#     10. two lines intersect externally, both lines are not parallel, 1 line has inf slope, 1 line has 0 slope

# case 1 (i)
class TestCase1i():
    line1 = (-2, 1, -2, 3)
    line2 = (-6, 2, -6, 3)
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test1i = TestCase1i()

class TestCase2i():
    line1 = (0, 2, 2, 0)
    line2 = (-3, 0, 0, -3)
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test2i = TestCase2i()


class TestCase3i():
    line1 = (-2, -1, 1, 2)
    line2 = (-4, 5, 5, -4)
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test3i = TestCase3i()



class TestCase3ii():
    line1 = (-10, -1, 5, 2)
    line2 = (-4, 6, 4, -2)
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test3ii = TestCase3ii()



class TestCase4i():
    line1 = (-10, -1, 5, 2)
    line2 = (-4, 6, -2, 4)
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)
#         intersection = compute_intersection_single_element(*self.line2, *self.line1, verbose=True)
#         print(intersection)

test4i = TestCase4i()

class TestCase4ii():
    line1 = (-10, -1, 5, 2)
    line2 = (-2, 4, -4, 6) # reverse the order of points
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)
#         intersection = compute_intersection_single_element(*self.line2, *self.line1, verbose=True)
#         print(intersection)

test4ii = TestCase4ii()

class TestCase5i():
    line1 = (-10, -1, 15, 4)
    line2 = (5, 4, 5, 0) # inf slope
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test5i = TestCase5i()

class TestCase5ii():
    line1 = (5, 4, 5, 0) # inf slope
    line2 = (-10, -1, 15, 4)
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test5ii = TestCase5ii()

class TestCase6i():
    line1 = (-10, -1, 0, 1)
    line2 = (5, 4, 5, 0) # inf slope
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test6i = TestCase6i()

class TestCase6ii():
    line1 = (5, 4, 5, 0) # inf slope
    line2 = (-10, -1, 0, 1)
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test6ii = TestCase6ii()

TestCase1i
(0.0, -8000000.0)
TestCase2i
(30000000.0, -30000000.0)
TestCase3i
(-0.0, 1.0000000185185187)
TestCase3ii
(0.8333333391203704, 1.1666666747685186)
TestCase4i
(0.8333333564814821, 1.1666666990740748)
TestCase4ii
(0.8333333101851859, 1.1666666342592602)
TestCase5i
(5.000000050000001, 2.0000000200000003)
TestCase5ii
(4.99999995, 1.9999999800000003)
TestCase6i
(5.000000125000003, 2.000000050000001)
TestCase6ii
(4.999999875000004, 1.9999999500000014)


In [35]:
# Test cases
#     7. two lines intersect internally, both lines are not parallel, one line has 0 slope
#     8. two lines intersect externally, both lines are not parallel, one line has 0 slope
#     9. two lines intersect internally, both lines are not parallel, 1 line has inf slope, 1 line has 0 slope
#     10. two lines intersect externally, both lines are not parallel, 1 line has inf slope, 1 line has 0 slope


class TestCase7i():
    line1 = (-2, -3, 4, 9)
    line2 = (5, 4, 0, 4) # 0 slope
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test7i = TestCase7i()

class TestCase7ii():
    line1 = (5, 4, 0, 4) # 0 slope
    line2 = (-2, -3, 4, 9)
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test7ii = TestCase7ii()

class TestCase8i():
    line1 = (-2, -3, 0, 1)
    line2 = (5, 4, 0, 4) # 0 slope
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test8i = TestCase8i()


class TestCase9i():
    line1 = (-2, 4, 6, 4)
    line2 = (3, 6, 3, -4) # 0 slope
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test9i = TestCase9i()

class TestCase10i():
    line1 = (-2, 4, 0, 4)
    line2 = (3, 6, 3, -4) # 0 slope
    
    def __init__(self):
        print(type(self).__name__)
        intersection = compute_intersection_single_element(*self.line1, *self.line2, verbose=True)
        print(intersection)

test10i = TestCase10i()

TestCase7i
(1.4999999750000004, 3.9999999333333345)
TestCase7ii
(1.5000000250000003, 4.000000066666668)
TestCase8i
(1.4999999250000036, 3.9999998000000097)
TestCase9i
(3.0000000375000004, 4.000000050000001)
TestCase10i
(3.0000001500000075, 4.00000020000001)
